In [2]:
import os
import sys
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio import Phylo
import numpy as np
import pandas as pd
import seaborn as sns
import pysam
import math
from Bio import Entrez
import pickle
mpl.rcParams['pdf.fonttype'] = 42
plt.rcParams['svg.fonttype'] = 'none'
%matplotlib inline

In [3]:
####
#USER INPUT
unmappedDir = '12_bwa_unmapped/'
blastDir = '13_bwa_blast/'
prokkaDir = '09_prokka_biopython/'
annoDir = '14_bwa_annotations'
subDir = 'subA/'
mappedID = 'D17-102065'

In [4]:
def checkSlash(directory):
    if directory[-1] != '/':
        directory = directory + '/'
    return directory

unmappedDir = checkSlash(unmappedDir)
blastDir = checkSlash(blastDir)
prokkaDir = checkSlash(prokkaDir)
annoDir = checkSlash(annoDir)
subDir = checkSlash(subDir)
gbDir = blastDir + 'gbRefs/'
if not os.path.exists(blastDir):
    os.makedirs(blastDir)
if not os.path.exists(annoDir):
    os.makedirs(annoDir)
if not os.path.exists(gbDir):
    os.makedirs(gbDir)

In [4]:
#Returns pandas dataframe of variable read mappings for a single sample of source reads
#Input = mappedID: source reads for mapping are from this sample
#        unmapped: directory of unmapped results
#Columns = samples in subgroup, plus 'sequence_1' and 'sequence_2' for the fwd & rev sequences of the read
#Rows = Read IDs
#Data = one if even one read mapped, zero if both paired ends are unmapped
def binaryMapping(mappedID, unmapped):
    unmappedDict = {}
    unmappedSeqs = {}
    for f in os.listdir(unmapped):
        if 'mapped_' + mappedID in f:
            contig = f.split('_')[1]
            unmappedDict[contig] = {}
            inFile = open(unmapped + f, 'r')
            for line in inFile:
                line = line.strip().split('\t')
                readID = line[0]
                flag = int(line[1])
                seq = line[9]
                if (flag & 64) and (flag & 4):
                    unmappedSeqs[readID + '__1'] = seq
                    unmappedDict[contig][readID + '__1'] = 10
                elif (flag & 128) and (flag & 4):
                    unmappedSeqs[readID + '__2'] = seq
                    unmappedDict[contig][readID + '__2'] = 10
            inFile.close()
    df = pd.DataFrame.from_dict(unmappedDict)
    df = df.fillna(value=1)             #1 if mapped
    df = df.replace(10, 0)              #0 if unmapped
    ####
    #Remove reads that are unmapped (e.g. zeros) in all samples
    df = df.loc[(df != 0).any(axis = 1)]
    #Remove reads that are mapped (e.g. ones) in all samples
    df = df.loc[(df.sum(axis=1) != len(df.columns))]
    df['sequence'] = ''
    for i in list(df.index):
        df['sequence'][i] = unmappedSeqs[i]
    return df

In [39]:
####
#For each mapped sample in a subgroup,
#import dataframe with reads as indexes and samples as columns.
#1's indicate that at least one PE read mapped, 0's mean both paired ends were unmapped
outSeqDict = {}
for f in os.listdir(unmappedDir + subDir):
    if 'mapped_' + mappedID in f:
        mappedSamp = f.split('_')[1]
    else:
        continue
    binaryFile = blastDir + subDir + 'mapped_' + mappedSamp + '_binary.txt'
    if os.path.exists(binaryFile):
        df = pd.DataFrame.from_csv(binaryFile, sep = '\t')
    else:
        df = binaryMapping(mappedSamp, unmappedDir + subDir)
        df.to_csv(binaryFile, sep = '\t')
    for i in list(df.index):
        if i not in outSeqDict:
            outSeqDict[i] = df['sequence'][i]

####
#Export reads of interest for BLAST analysis
outFile = open(blastDir + subDir + 'differentially_mapped_reads.fa', 'w')
for i in outSeqDict:
    outFile.write('>' + i + '\n')
    outFile.write(outSeqDict[i] + '\n')
outFile.close()

# Read matches to nt database

$ update_blastdb.pl --passive --timeout 300 --force --verbose nr &> nr.updatedb.log

In [34]:
def runBlast(inFile, threads):
    if not os.path.exists(inFile.replace('.fa', '.out')):
        os.system('blastn -task blastn -db nt_blast/nt -query ' + inFile + ' -out ' + \
                  inFile.replace('.fa', '.out') + ' -num_threads ' + str(threads) + \
                 ' -outfmt 6 -max_target_seqs 1')

def bestBlast(blastFile, outFile):
    f = open(blastFile, 'r')
    o = open(outFile, 'w')
    reads = set()
    data = {}
    for line in f:
        lineList = line.strip().split('\t')
        readID = lineList[0]
        if readID in reads: #Keep only top BLAST hit
            continue
        reads.add(readID)
        o.write(line)
    f.close()
    o.close()

#Write file to download genbank references
#Check for existing genbank files in a shared directory and skip downloading those
def getAccessions(inFile, gbDir, outFile):
    gbDatabase = set()
    for i in os.listdir(gbDir):
        gbDatabase.add(i.replace('.gb', ''))
    f = open(inFile, 'r')
    giSet = set()
    for line in f:
        line = line.strip().split('\t')
        gi = line[1].split('|')[1]
        gb = line[1].split('|')[3].split('.')[0]
        if gb not in gbDatabase:
            giSet.add(gi)
    f.close()
    o = open(outFile, 'w')
    for gi in giSet:
        o.write(gi + '\n')
    o.close()
    
def divideBlastFile(inFile, chunkDir, chunks):
    inHandle = inFile.replace('.out', '')
    f = open(inFile, 'r')
    chunks = int(chunks)
    i = 0
    j = 0
    outF = open(chunkDir + str(j) + '.out', 'w')
    for line in f:
        if (i != 0) and (i % chunks == 0):
            outF.close()
            j +=1
            outF = open(chunkDir + str(j) + '.out', 'w')
        outF.write(line)
        i += 1
    f.close()
    outF.close()

In [35]:
####
#BLAST sequences of interest against the nt database
diffFile = blastDir + subDir + 'differentially_mapped_reads.fa'
runBlast(diffFile, 30)

In [23]:
####
#Write the best BLAST matches to a file
blastFile = blastDir + subDir + 'differentially_mapped_reads.out'
outFile = blastDir + subDir + 'differentially_mapped_best.out'
bestBlast(blastFile, outFile)

In [35]:
####
#Export BLAST GI numbers to a separate file for genbank download
blastFile = blastDir + subDir + 'differentially_mapped_best.out'
accessionFile = blastDir + subDir + 'differentially_mapped_best_accessions.txt'
getAccessions(blastFile, gbDir, accessionFile)

In [26]:
####
#Download genbank files for all gi numbers recovered
#Consider running this in the background
commandString = 'python scripts/getGenbank.py -i ' + accessionFile + ' -d nuccore -e sjspen@gmail.com -o ' + gbDir
print('Run this in the background:\n')
print(commandString)
#os.system(commandString)

#NOTE: may need to check resulting files for any that have a locus different than accession/version,
#didn't know how to fix other guys code.
#Could probably fix by editing the getGenbank script right when the full string is written - grab accession from there

#$ mv HUMRSALPC.gb M28031.gb
#$ mv HUAC002038.gb AC002038.gb

Run this in the background:

python scripts/getGenbank.py -i 13_bwa_blast/subA/differentially_mapped_best_accessions.txt -d nuccore -e sjspen@gmail.com -o 13_bwa_blast/gbRefs/


0

In [29]:
####
#Divide BLAST file into parallel chunks
inFile = blastDir + subDir + 'differentially_mapped_best.out'
chunkDir = blastDir + subDir + 'blast_chunks/'
if not os.path.exists(chunkDir):
    os.makedirs(chunkDir)
chunks = 1000
divideBlastFile(inFile, chunkDir, chunks)

In [33]:
####
#Run BLAST annotations in parallel on chunked BLAST files
chunkDir = blastDir + subDir + 'blast_chunks/'
chunkDir = checkSlash(chunkDir)
batchFile = blastDir + subDir + 'annotationBatch.sh'
parallel = 16
b = open(batchFile, 'w')
b.write('#!/bin/bash\n#Annotation commands\n\n')
for f in os.listdir(chunkDir):
    f = chunkDir + f
    o = f.replace('.out', '_annotation.txt')
    command = './scripts/gbFeatures.py -i ' + f + ' -g ' + gbDir + ' -o ' + o + '\n'
    b.write(command)
b.close()
os.system('chmod +x ' + batchFile)
print('Run this command in the background:')
print('cat ' + batchFile + ' | parallel -j ' + str(parallel))
#os.system('cat ' + batchFile + ' | parallel -j ' + str(parallel))

Run this command in the background:
cat 13_bwa_blast/subA/annotationBatch.sh | parallel -j 16


In [ ]:
#These are the other features that could be recovered in different files!
set(['misc_feature', 'tRNA', 'repeat_region', 'rRNA', 'exon', 'misc_RNA', 'gene'])
set(['misc_feature', 'tRNA', 'sig_peptide', 'rRNA', 'exon', 'mRNA', 'gene', 'ncRNA'])
set(['misc_feature', 'tRNA', 'repeat_region', 'rRNA', 'exon', 'mRNA', 'gene', 'ncRNA'])
set(['misc_feature', 'misc_recomb', 'tRNA', 'rRNA', 'exon', 'gene'])
set(['misc_feature', 'unsure', 'tRNA', 'mRNA', 'misc_RNA', 'gene'])
set(['misc_feature', 'tRNA', 'repeat_region', 'sig_peptide', 'rRNA', 'mRNA', 'misc_RNA', 'gene', 'ncRNA'])
set(['misc_feature', 'tRNA', 'repeat_region', 'sig_peptide', 'rRNA', 'gene', 'ncRNA'])
set(['misc_feature', 'unsure', 'tRNA', 'repeat_region', 'sig_peptide', 'rRNA', 'gene'])
set(['misc_feature', 'tRNA', 'repeat_region', 'rRNA', 'misc_RNA', 'gene', 'ncRNA'])
set(['misc_feature', 'tRNA', 'repeat_region', 'rRNA', 'exon', 'mRNA', 'STS', 'gene'])
set(['misc_feature', 'tRNA', 'repeat_region', 'sig_peptide', 'rRNA', 'assembly_gap', 'misc_RNA', 'gene', 'ncRNA'])
set(['misc_feature', 'tRNA', 'repeat_region', 'rRNA', 'mRNA', 'gene', 'ncRNA'])

In [36]:
####
#Concatenate annotation results
#Includes 'CDS' annotations and 'source' annotations, in case the source is useful (many from FRC)
annoFile = blastDir + subDir + 'differentially_mapped_best_coding.txt'
chunkDir = blastDir + subDir + 'blast_chunks/'
chunkDir = checkSlash(chunkDir)
catString = 'cat '
for f in os.listdir(chunkDir):
    if '_annotation.txt' in f:
        f = chunkDir + f
        catString += f + ' '
catString += ' > ' + annoFile
os.system(catString)

0

In [49]:
def unpackQuals(qualString):
    qualList = qualString.split('__')
    qualDict = {}
    for i in qualList:
        i = i.split('||')
        qualDict[i[0]] = i[1]
    return qualDict

def summarizeHits(binaryFile, annoFile, outFile):
    #Import annotations
    df = pd.DataFrame.from_csv(binaryFile, sep='\t')
    annoDict = {}
    inFile = open(annoFile,'r')
    for line in inFile:
        line = line.strip().split('\t')
        readID = line[0]
        if readID in list(df.index):
            if line[3] == 'CDS':
                if readID in annoDict:
                    print(readID)
                    print('multiple cds')
                annoDict[readID] = unpackQuals(line[7])
    inFile.close()
    annoDf = pd.DataFrame.from_dict(annoDict)
    annoDf = annoDf.T
    #Export sums of annotations on different contigs
    sumDict = {}
    for s in df:
        if s == 'sequence':
            continue
        for i in list(df.index):
            if df[s][i] == 1:
                if s not in sumDict:
                    sumDict[s] = {}
                if i in list(annoDf.index):
                    product = annoDf['product'][i]
                else:
                    product = 'N/A'
                if product not in sumDict[s]:
                    sumDict[s][product] = 0
                sumDict[s][product] += 1
    sumDf = pd.DataFrame.from_dict(sumDict)
    sumDf.to_csv(outFile, sep = '\t')
    return sumDf

In [50]:
####
#Output a summary of CDS hits for each binary mapping in a pandas-compatible text file
binaryFile = blastDir + subDir + 'mapped_D17-102065_binary.txt'
annoFile = blastDir + subDir + 'differentially_mapped_best_coding.txt'
outFile = blastDir + subDir + 'mapped_D17-102065_summary.txt'

sumDf = summarizeHits(binaryFile, annoFile, outFile)
sumDf

NB501288_148_H2MVCBGX2:2:22203:14909:16420#GGACTCCTGGGCTTAG__1
multiple cds


,D17-102036,D17-102037,D17-102038,D17-102039,D17-102040,D17-102041,D17-102042,D17-102043,D17-102044,D17-102045,D17-102046,D17-102047,D17-102048,D17-102049,D17-102050,D17-102051,D17-102065
1-pyrroline-5-carboxylate dehydrogenase,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
"2,5-didehydrogluconate reductase",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
2-oxoacid:ferredoxin oxidoreductase subunit beta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
"4Fe-4S iron sulfur cluster binding s, NifH/frxC family protein",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6-phosphogluconolactonase,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0
ABC transporter ATP-binding protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
ACP S-malonyltransferase,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
ATP-dependent Clp protease ATP-binding subunit ClpA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
ATP-dependent helicase HrpB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Acyl-CoA dehydrogenase,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0


# Read placements within de novo assemblies

In [51]:
#reads: list of reads to annotate
#alignmentFile: alignment to contigs
#prokkaFile: prokka-annotated contigs
def importAlignment(reads, alignmentFile):
    annoDf = pd.DataFrame(index = reads, columns = ['ref_id', 'ref_pos'])
    sam = pysam.AlignmentFile(alignmentFile, "r")
    for r in sam:
        if r.flag & 4:
            continue
        refName = sam.get_reference_name(r.tid)
        if (r.qname + '__1' in reads) and (r.flag & 64):
            annoDf['ref_id'][r.qname + '__1'] = refName
            annoDf['ref_pos'][r.qname + '__1'] = r.pos
        elif (r.qname + '__2' in reads) and (r.flag & 128):
            annoDf['ref_id'][r.qname + '__2'] = refName
            annoDf['ref_pos'][r.qname + '__2'] = r.pos
    return annoDf

def getAnnotation(prokkaFile, node, pos):
    records = list(SeqIO.parse(prokkaFile, 'genbank'))
    passedMapping = False
    for r in records:
        if r.id == node:
            prevFeature = np.nan
            for f in r.features:
                if f.type == 'source':
                    continue
                start = str(f.location).split(':')[0].replace('[', '')
                end = str(f.location).split(':')[1].split(']')[0]
                strand = str(f.location).split('(')[1].replace(')', '')
                #Look for a coding feature
                if (pos > int(start)) and (pos < int(end)):
                    return {'coding': f}
                #Look for intergenic
                if (pos < int(start)) and (pos < int(end)):
                    if isinstance(prevFeature, float):
                        prevStrand = '+'
                    else:
                        prevStrand = str(prevFeature.location).split('(')[1].replace(')', '')
                    if (prevStrand == '+') and (strand == '+'):
                        return {'downstream1': prevFeature, 'upstream1': f}
                    elif (prevStrand == '-') and (strand == '-'):
                        return {'downstream1': f, 'upstream1': prevFeature}
                    elif (prevStrand == '+') and (strand == '-'):
                        return {'upstream1': prevFeature, 'upstream2': f}
                    else:
                        return {'downstream1': prevFeature, 'downstream2': f}
                #store prev
                prevFeature = f

def summarizeAnnotation(featureDf, i, featureDict, flag):
    if flag in featureDict:
        if isinstance(featureDict[flag], float):
            return featureDf
        quals = featureDict[flag].qualifiers
        if 'gene' in quals:
            featureDf[flag][i] = quals['gene'][0]
        elif 'product' in quals:
            featureDf[flag][i] = quals['product'][0]
        else:
            print('No gene/product:')
            print(quals)
    return featureDf

In [ ]:
for f in os.listdir(unmappedDir + subDir):
    if 'mapped_' + mappedID in f:
        contigID = f.split('_')[1]
    else:
        continue
    if os.path.exists(annoDir + mappedID + '_' + contigID + '_annotations.txt'):
        continue
    prokkaFile = prokkaDir + contigID + '.gb'
    annoDf = importAlignment(list(df.index), '11_bwa/subA/contig_' + contigID + '_mapped_' + mappedID + '.sam')
    annoDf['coding'] = np.nan
    annoDf['downstream1'] = np.nan
    annoDf['downstream2'] = np.nan
    annoDf['upstream1'] = np.nan
    annoDf['upstream2'] = np.nan
    for i in list(annoDf.index):
        if isinstance(annoDf['ref_id'][i], str):
            featureDict = getAnnotation(prokkaFile, annoDf['ref_id'][i], annoDf['ref_pos'][i])
        else:
            continue
        if not isinstance(featureDict, dict): #protect from null values, no annotation
            continue
        annoDf = summarizeAnnotation(annoDf, i, featureDict, 'coding')
        annoDf = summarizeAnnotation(annoDf, i, featureDict, 'downstream1')
        annoDf = summarizeAnnotation(annoDf, i, featureDict, 'downstream2')
        annoDf = summarizeAnnotation(annoDf, i, featureDict, 'upstream1')
        annoDf = summarizeAnnotation(annoDf, i, featureDict, 'upstream2')
    annoDf.to_csv(annoDir + mappedID + '_' + contigID + '_annotations.txt', sep='\t')

/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_1_length_318532_cov_30.8598 318532 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_2_length_294520_cov_33.7766 294520 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_3_length_274993_cov_27.222 274993 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LO

/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_41_length_49499_cov_30.8857 49499 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_42_length_49295_cov_31.5507 49295 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_43_length_49237_cov_33.501 49237 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS

/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_1_length_453946_cov_48.2576 453946 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_2_length_415049_cov_49.2747 415049 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malform

/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_28_length_78503_cov_43.6146 78503 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_29_length_76161_cov_46.3782 76161 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_30_length_73199_cov_42.6729 73199 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCU

/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_1_length_315936_cov_36.4331 315936 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_2_length_294264_cov_42.3093 294264 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_3_length_259367_cov_39.0721 259367 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed L

...

/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_1_length_415050_cov_38.4749 415050 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_2_length_318539_cov_36.6346 318539 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_3_length_315936_cov_33.1247 315936 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed L

/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_1_length_282643_cov_34.6171 282643 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_2_length_244214_cov_29.2389 244214 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_3_length_232738_cov_32.3255 232738 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed L

/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_33_length_77446_cov_34.54 77446 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_34_length_76771_cov_30.5378 76771 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_35_length_73222_cov_30.4016 73222 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS 

/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_1_length_410436_cov_31.8567 410436 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_2_length_339451_cov_40.2707 339451 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_3_length_315936_cov_33.7714 315936 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed L

/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_32_length_63862_cov_41.81 63862 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_33_length_59523_cov_37.4874 59523 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       NODE_34_length_58571_cov_33.6449 58571 bp DNA BCT linear 07-APR-2017\n'
  "correct?\n:%r" % line, BiopythonParserWarning)
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/Bio/GenBank/Scanner.py:1250: BiopythonParserWarning: Malformed LOCUS 

In [14]:
def quantifyAnnotations(inFile):
    print(inFile)
    annoDf = pd.DataFrame.from_csv(inFile, sep = '\t')
    print(annoDf['downstream1'].value_counts())
    print('')
    
#Summarize annotations
for f in os.listdir(unmappedDir + subDir):
    if 'mapped_' + mappedID in f:
        contigID = f.split('_')[1]
    else:
        continue
    inFile = annoDir + mappedID + '_' + contigID + '_annotations.txt'
    quantifyAnnotations(inFile)

14_bwa_annotations/D17-102065_D17-102037_annotations.txt
hypothetical protein    91
tRNA-Lys                 5
tRNA-Leu                 4
tRNA-Ala                 4
betB_2                   3
tRNA-Gly                 3
23S ribosomal RNA        2
dmlR_22                  2
rhsC_5                   1
pobB                     1
hcpA_3                   1
Name: downstream1, dtype: int64

14_bwa_annotations/D17-102065_D17-102047_annotations.txt
phoB_2                  18
tRNA-Lys                 7
tRNA-Leu                 4
tRNA-Gly                 4
tRNA-Ala                 4
betB_1                   3
hypothetical protein     3
23S ribosomal RNA        2
dmlR_23                  2
rhsC_7                   1
pobB                     1
hcpA_2                   1
Name: downstream1, dtype: int64

14_bwa_annotations/D17-102065_D17-102044_annotations.txt
hypothetical protein    97
phoB_2                  18
tRNA-Lys                 5
tRNA-Leu                 4
tRNA-Gly                 3
dmlR_11

14_bwa_annotations/D17-102065_D17-102037_annotations.txt
hypothetical protein    91
tRNA-Lys                 5
tRNA-Leu                 4
tRNA-Ala                 4
betB_2                   3
tRNA-Gly                 3
23S ribosomal RNA        2
dmlR_22                  2
rhsC_5                   1
pobB                     1
hcpA_3                   1
Name: downstream1, dtype: int64

14_bwa_annotations/D17-102065_D17-102047_annotations.txt
phoB_2                  18
tRNA-Lys                 7
tRNA-Leu                 4
tRNA-Gly                 4
tRNA-Ala                 4
betB_1                   3
hypothetical protein     3
23S ribosomal RNA        2
dmlR_23                  2
rhsC_7                   1
pobB                     1
hcpA_2                   1
Name: downstream1, dtype: int64

14_bwa_annotations/D17-102065_D17-102044_annotations.txt
hypothetical protein    97
phoB_2                  18
tRNA-Lys                 5
tRNA-Leu                 4
tRNA-Gly                 3
dmlR_11

# Discarded code

In [ ]:
split = 1000
diffFile = blastDir + subDir + 'differentially_mapped_reads.fa'
j = 0
outFileName = diffFile.replace('.fa', '_' + j + '.fa')
with open(diffFile, 'r') as f:
    i = 0
    for line in f:
        if (i != 0) and (i % 1000 == 0):
            outFile.close()
            outFileName = outFileName.replace('_' + str(j) + '.fa', '_' + str(j+1) + '.fa')
            outFile = open(outFileName, 'w')
            j += 1
        
        i += 1

####
#Efficiently store features by location
for inFile in os.listdir(gbDir):
    df = pd.DataFrame(columns = ['end', 'strand', 'type', 'qualifiers'])
    records = list(SeqIO.parse(inFile, 'genbank'))
    for r in records:
        if f.type == 'source':
            continue
        if f.type == 'CDS':
            
        else:
            print(f.type)
        
        prevFeature = np.nan
        for f in r.features:
            if f.type == 'source':
                continue
            start = str(f.location).split(':')[0].replace('[', '')
            end = str(f.location).split(':')[1].split(']')[0]
            strand = str(f.location).split('(')[1].replace(')', '')

####
#Summarize blast hits by extracting features from downloaded genbank files
#Export to tab-delimited file matching readIDs to coding/upstream/downstream gene names
blastFile = blastDir + subDir + 'differentially_mapped_reads.out'
outFile = blastDir + subDir + 'differentially_mapped_reads.txt'
commandString = './scripts/gbFeatures.py -i ' + blastFile + ' -g ' + gbDir + ' -o ' + outFile
print('Run this command in the background:\n')
print(commandString)